In [1]:
import open3d as o3d
import os
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import re
import os
import random
import csv

from mpl_toolkits.mplot3d import Axes3D


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Parse Arguments / Define Constants

In [ ]:
RAW_PCD_FOLDER = 'label_cloud_project/datastore/pointclouds/raw_pcds'
CROPPED_PCD_FOLDER = 'label_cloud_project/datastore/pointclouds/cropped_pcds'  
LABEL_CLOUD_LABEL_FOLDER = 'label_cloud_project/datastore/labels/label_cloud_labels'
YOLO_LABEL_FOLDER = 'label_cloud_project/datastore/labels/yolo_labels'  
BEV_IMAGE_FOLDER = 'label_cloud_project/datastore/images/birds_eye_view_images'

# Run Inference Continuously

In [ ]:
import os
import time
import cv2
import numpy as np
import model_training.config as config
import model_training.utils as utils
import inference.single_image_inference as inference
import preprocessing.preprocess_data as preprocessing
from datetime import datetime
from model_training.model import YOLOv3

current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
run_id = f"{config.INFERENCE_RUN_TITLE}_{current_datetime}"

# Create a folder for the run_id inside 'inference/inference_results/'
results_folder = os.path.join(config.INFERENCE_RESULTS_FOLDER, run_id)
processed_pcd_folder = config.INFERENCE_PROCESSED_PCD_FOLDER

os.makedirs(results_folder, exist_ok=True)

model = YOLOv3(num_classes=config.NUM_CLASSES).to(config.DEVICE)
utils.load_checkpoint(
            config.INFERENCE_CHECKPOINT_FILE, model, None, None
        )
# Inference loop
while True:
    # Monitor the folder for new point cloud files
    for file_name in os.listdir(config.INFERENCE_PCD_FOLDER):
        if file_name.endswith(".pcd"):
            # Transform point cloud to BEV image
            bev_image = preprocessing.transform_to_bev(file_name)

            # Run BEV model inference
            bev_output = model.predict(bev_image)

            # Extract x, y coordinates from YOLO v3 output
            confidence_score, x, y, w, h = inference.inference_single_image(model, bev_output)

            # Print results to console
            print(f"File: {file_name}, confidence: {confidence_score}, X: {x}, Y: {y}, W: {w}, H: {h}")

            # Write results to text file inside the run_id folder
            results_file_path = os.path.join(results_folder, "results.txt")
            with open(results_file_path, "a") as f:
                f.write(f"File: {file_name}, confidence: {confidence_score}, X: {x}, Y: {y}, W: {w}, H: {h}\n")

            # Optional: Move processed file to another folder to avoid re-processing
            os.rename(os.path.join(config.INFERENCE_PCD_FOLDER, file_name), os.path.join(processed_pcd_folder, file_name))

    # Sleep for a while before checking for new files again
    time.sleep(1)